Calculate ensemble of sea level projections

In [ ]:
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt

import sys
sys.path.append('../src')

from alldata import AllData
from ensemble import EnsembleMember,FullEnsemble

np.set_printoptions(precision=2)
%matplotlib notebook
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

Read data

In [ ]:
ad = AllData()
ad.year0 = 1951
ad.usefanom = True
ad.gather(update=False)

#Omit invalid ism
ad.ds = ad.ds.drop_sel(ism='PISM_DMI')

#Optionally subsection of models for rapid calculation
#ad.ds = ad.ds.isel(ism=slice(0,2),esm=slice(0,2))
#ad.ds = ad.ds.isel(esm=slice(0,5))


fe = FullEnsemble(ad)
fe.cal = 'S'
fe.nonlin = 'cutoff'
fe.lump = True
fe.gather(force_update=False)

Figure settings

In [ ]:
plt.style.use('../src/style_paper')
mpl.rc('figure.subplot',left=.15,bottom=.2)

alpha = .3 #Transparency of uncertainty range
perc = [17,83] #Percentiles of uncertainty range
titles = ['SSP1-2.6','SSP2-4.5','SSP5-8.5']

Make and save figure

In [ ]:
fig,ax = plt.subplots(1,3,sharex=True,sharey='row',figsize=(7,3))

print(np.sum(~np.isnan(fe.gamma[0,:,:].values)),'of',fe.gamma.shape[1]*fe.gamma.shape[2],'valid gammas')    

for s,ss in enumerate(ad.ssp):
    
    #Plot no feedback
    ax[s].plot(fe.time,100*np.nanmedian(fe.slr[0,s,:,:,:],axis=(0,1)),c='.5',lw=2,ls='--',label='No feedback')
    ax[s].fill_between(fe.time,100*np.nanpercentile(fe.slr[0,s,:,:,:],perc[0],axis=(0,1)),100*np.nanpercentile(fe.slr[0,s,:,:,:],perc[1],axis=(0,1)),color='.5',alpha=alpha)#,label=perc)

    #Plot with feedback
    ax[s].plot(fe.time,100*np.nanmedian(fe.slr[1,s,:,:,:],axis=(0,1)),c=ad.scol[ss],lw=2,label='Feedback')
    ax[s].fill_between(fe.time,100*np.nanpercentile(fe.slr[1,s,:,:,:],perc[0],axis=(0,1)),100*np.nanpercentile(fe.slr[1,s,:,:,:],perc[1],axis=(0,1)),color=ad.scol[ss],alpha=alpha)
    
    ax[s].set_title(titles[s])
    
    print(ss,'md',np.nanmedian(fe.slr[1,s,:,:,:],axis=(0,1))[-1]/(np.nanmedian(fe.slr[0,s,:,:,:],axis=(0,1))[-1]))
    #print(ss,'mn',np.nanmean(fe.slr[1,s,:,:,:],axis=(0,1))[-1]/(np.nanmean(fe.slr[0,s,:,:,:],axis=(0,1))[-1]))
    print(ss,'83',np.nanpercentile(fe.slr[1,s,:,:,:],83,axis=(0,1))[-1]/(np.nanpercentile(fe.slr[0,s,:,:,:],83,axis=(0,1))[-1]))
    #print(ss,'95',np.nanpercentile(fe.slr[1,s,:,:,:],95,axis=(0,1))[-1]/(np.nanpercentile(fe.slr[0,s,:,:,:],95,axis=(0,1))[-1]))

    
ax[0].legend(loc='upper left')
ax[0].set_ylabel('Cumulative sea-level rise [cm]')
ax[1].set_xlabel('Time [years]')

plt.savefig(f'../draftfigs/full_ensemble_cal{fe.cal}_{fe.nonlin}_{fe.year0}_lump.png',dpi=450,facecolor='w',transparent=False)
plt.show()